In [1]:
import logging
from typing import Tuple, Dict, Any
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from sklearn.metrics import (
    roc_curve, auc, confusion_matrix, accuracy_score,
    precision_recall_fscore_support, precision_score,
    recall_score, f1_score, roc_auc_score
)
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from peft import get_peft_model, LoraConfig, TaskType

/home/dcli/miniconda3/envs/malicious_prompt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Constants
MODEL_NAME = "microsoft/deberta-v3-small"
MAX_LENGTH = 256
BATCH_SIZE = 8
NUM_EPOCHS = 100
WEIGHT_DECAY = 0.01
MODEL_SAVE_PATH = "./deberta_lora_classifier_full"

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("DeBERTa Initial Runs")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

class PromptClassifier:
    def __init__(self):
        """Initialize the classifier with model, tokenizer, and LoRA config."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            MODEL_NAME, use_fast=False
        )
        self.base_model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME, num_labels=2
        ).to(device)

        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1
        )
        self.model = get_peft_model(self.base_model, lora_config).to(device)
        logger.info("DeBERTa LoRA model initialized.")

    def preprocess_data(self, examples: Dict[str, Any]) -> Dict[str, Any]:
        """Preprocess the input data using the tokenizer."""
        return self.tokenizer(
            examples["Prompt"],
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH
        ).to(device)

    def prepare_datasets(self, df: pd.DataFrame) -> Tuple[Dataset, Dataset]:
        """Prepare training and validation datasets."""
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            df["Prompt"].tolist(),
            df["Malicious (0/1)"].tolist(),
            test_size=0.2,
            random_state=42
        )

        train_data = Dataset.from_dict({
            "Prompt": train_texts,
            "label": train_labels
        })
        val_data = Dataset.from_dict({
            "Prompt": val_texts,
            "label": val_labels
        })

        train_data = train_data.map(self.preprocess_data, batched=True)
        val_data = val_data.map(self.preprocess_data, batched=True)

        return train_data, val_data

    def compute_metrics(
        self, eval_pred: Tuple[np.ndarray, np.ndarray]
    ) -> Dict[str, float]:
        """Compute evaluation metrics."""
        logits, labels = eval_pred
        predictions = torch.argmax(torch.tensor(logits), dim=-1)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average='binary'
        )
        acc = accuracy_score(labels, predictions)
        return {
            "accuracy": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }

    def train(self, train_data: Dataset, val_data: Dataset) -> None:
        """Train the model."""
        training_args = TrainingArguments(
            output_dir="./results",
            eval_strategy="epoch",
            save_strategy="epoch",
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            num_train_epochs=NUM_EPOCHS,
            weight_decay=WEIGHT_DECAY,
            logging_dir="./deberta_logs",
            logging_strategy="steps",
            logging_steps=50,
            save_total_limit=3,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            evaluation_strategy="epoch",
            report_to="none"
        )

        early_stopping_callback = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.0
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=val_data,
            compute_metrics=self.compute_metrics,
            callbacks=[early_stopping_callback]
        )

        trainer.train()
        self.save_model()

    def save_model(self) -> None:
        """Save the trained model and tokenizer."""
        self.model.save_pretrained(MODEL_SAVE_PATH)
        self.tokenizer.save_pretrained(MODEL_SAVE_PATH)
        logger.info(f"Model saved to {MODEL_SAVE_PATH}")

    def evaluate(
        self, val_data: Dataset
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Evaluate the model and return predictions."""
        test_trainer = Trainer(self.model)
        predictions = test_trainer.predict(val_data)

        logits = torch.tensor(predictions.predictions)
        probs = torch.nn.functional.softmax(logits, dim=-1).numpy()
        pred_labels = np.argmax(probs, axis=-1)

        return predictions.label_ids, pred_labels, probs[:, 1]

    def plot_metrics(
        self, true_labels: np.ndarray, pred_labels: np.ndarray,
        probs: np.ndarray
    ) -> None:
        """Plot ROC curve and confusion matrix."""
        # ROC Curve
        fpr, tpr, _ = roc_curve(true_labels, probs)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(
            fpr, tpr, color="blue", lw=2,
            label=f"ROC Curve (AUC = {roc_auc:.2f})"
        )
        plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic (ROC) Curve")
        plt.legend(loc="lower right")
        plt.savefig(
            "roc_curve_lora_full.png",
            dpi=300,
            bbox_inches="tight"
        )
        plt.close()

        # Confusion Matrix
        cm = confusion_matrix(true_labels, pred_labels)
        plt.figure(figsize=(6, 5))
        sns.heatmap(
            cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["0", "1"], yticklabels=["0", "1"]
        )
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title("Confusion Matrix")
        plt.savefig(
            "confusion_matrix_lora_full.png",
            dpi=300,
            bbox_inches="tight"
        )
        plt.close()

2025-03-19 21:50:22,244 - INFO - Using device: cuda


In [5]:
# Load data
df = pd.read_csv(
    "./data/FINAL_validated_prompts_with_similarity_deBERTa.csv"
)
df["Malicious (0/1)"] = df["Malicious (0/1)"].astype(int)
"""
# Separate malicious and benign prompts
malicious_prompts = df[df["Malicious (0/1)"] == 1]
benign_prompts = df[df["Malicious (0/1)"] == 0]

# Randomly sample 3000 prompts from each class
malicious_sample = malicious_prompts.sample(n=3000, random_state=42)
benign_sample = benign_prompts.sample(n=3000, random_state=42)

# Combine the samples
balanced_df = pd.concat([malicious_sample, benign_sample])
# Shuffle the combined dataset
balanced_df = balanced_df.sample(frac=1, random_state=42) \
        .reset_index(drop=True)

logger.info(f"Created balanced dataset with {len(balanced_df)} rows")
logger.info(f"Number of malicious prompts: {sum(balanced_df['Malicious (0/1)'] == 1)}")
logger.info(f"Number of benign prompts: {sum(balanced_df['Malicious (0/1)'] == 0)}")
"""
# Initialize and train classifier
classifier = PromptClassifier()
train_data, val_data = classifier.prepare_datasets(df)

train_total = len(train_data['label'])
val_total = len(val_data['label'])
train_1_total = sum(train_data['label'])
val_1_total = sum(val_data['label'])
logger.info(f"Training: {train_1_total} malicious prompts")
logger.info(f"Training: {train_total-train_1_total} benign prompts")
logger.info(f"Validation: {val_1_total} malicious prompts")
logger.info(f"Validation: {val_total-val_1_total} benign prompts")
    
logger.info("Starting fine-tuning...")
classifier.train(train_data, val_data)

# Evaluate and plot results
logger.info("Evaluating...")
true_labels, pred_labels, probs = classifier.evaluate(val_data)

# Print metrics
print(f"Accuracy: {accuracy_score(true_labels, pred_labels):.4f}")
print(f"Precision: {precision_score(true_labels, pred_labels):.4f}")
print(f"Recall: {recall_score(true_labels, pred_labels):.4f}")
print(f"F1 Score: {f1_score(true_labels, pred_labels):.4f}")
print(f"AUC Score: {roc_auc_score(true_labels, probs):.4f}")

# Plot metrics
classifier.plot_metrics(true_labels, pred_labels, probs)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-03-19 21:50:26,802 - INFO - DeBERTa LoRA model initialized.
Map: 100%|██████████| 15365/15365 [00:01<00:00, 13889.37 examples/s]
2025-03-19 21:50:36,560 - INFO - Training: 30768 malicious prompts
2025-03-19 21:50:36,561 - INFO - Training: 30689 benign prompts
2025-03-19 21:50:36,561 - INFO - Validation: 7679 malicious prompts
2025-03-19 21:50:36,561 - INFO - Validation: 7686 benign prompts
2025-03-19 21:50:36,562 - INFO - Starting fine-tuning...
/home/dcli/miniconda3/envs/malicious_prompt_env/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.172000,0.164165,0.952099,0.951515,0.962805,0.940487
2,0.155200,0.157677,0.957631,0.958474,0.939360,0.978383
3,0.162000,0.138905,0.960950,0.960795,0.964197,0.957416
4,0.138300,0.135525,0.963619,0.963528,0.965481,0.961584
5,0.114800,0.150929,0.964335,0.964503,0.959531,0.969527
6,0.155700,0.137997,0.964139,0.963810,0.972303,0.955463
7,0.155600,0.122361,0.966612,0.966807,0.960777,0.972913
8,0.127500,0.123935,0.967459,0.967528,0.965021,0.970048
9,0.099000,0.134544,0.966287,0.966015,0.973423,0.958719
10,0.079800,0.131984,0.966938,0.967124,0.961276,0.973043


2025-03-19 22:28:01,746 - INFO - Model saved to ./deberta_lora_classifier_full
2025-03-19 22:28:01,747 - INFO - Evaluating...


Accuracy: 0.9666
Precision: 0.9608
Recall: 0.9729
F1 Score: 0.9668
AUC Score: 0.9951
